In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

import pandas as pd
import numpy as np

import plotly.express as ex
import plotly.graph_objects as go

In [2]:
app=dash.Dash(__name__)

In [9]:
df=pd.read_csv('./Datasets/intro_bees.csv')

In [10]:
df

,Program,Year,Period,State,ANSI,Affected by,Pct of Colonies Impacted,state_code
0,SURVEY,2019,JAN THRU MAR,Alabama,1,Disease,1.8,AL
1,SURVEY,2019,JAN THRU MAR,Alabama,1,Other,3.1,AL
2,SURVEY,2019,JAN THRU MAR,Alabama,1,Pesticides,0.3,AL
3,SURVEY,2019,JAN THRU MAR,Alabama,1,Pests_excl_Varroa,22.7,AL
4,SURVEY,2019,JAN THRU MAR,Alabama,1,Unknown,9.0,AL
...,...,...,...,...,...,...,...,...
4585,SURVEY,2015,OCT THRU DEC,Wyoming,56,Other,11.3,WY
4586,SURVEY,2015,OCT THRU DEC,Wyoming,56,Pesticides,10.5,WY
4587,SURVEY,2015,OCT THRU DEC,Wyoming,56,Pests_excl_Varroa,3.3,WY
4588,SURVEY,2015,OCT THRU DEC,Wyoming,56,Unknown,3.9,WY


In [11]:
df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()

In [12]:
df.reset_index(inplace=True)

In [13]:
df

,State,ANSI,Affected by,Year,state_code,Pct of Colonies Impacted
0,Alabama,1,Disease,2015,AL,0.050
1,Alabama,1,Disease,2016,AL,1.200
2,Alabama,1,Disease,2017,AL,2.250
3,Alabama,1,Disease,2018,AL,1.300
4,Alabama,1,Disease,2019,AL,1.800
...,...,...,...,...,...,...
1345,Wyoming,56,Varroa_mites,2015,WY,23.375
1346,Wyoming,56,Varroa_mites,2016,WY,19.475
1347,Wyoming,56,Varroa_mites,2017,WY,23.525
1348,Wyoming,56,Varroa_mites,2018,WY,22.425


In [23]:
app.layout=html.Div([
    html.H1('Application Dashboard with DASH',style={'text-align':'center'}),
    dcc.Dropdown(
    id='select_year',
    options=[{'label':'2015','value':2015},{'label':'2016','value':2016},{'label':'2017','value':2017},{'label':'2019','value':2019}],
    multi=False,
    value=2015,
    style={'width':'40%'}
    ),
    html.Div(id='output_container',children=[]),
    html.Br(),
    
    dcc.Graph(id='my_bee_map',figure={})
])

In [28]:
@app.callback(
[Output(component_id="output_container",component_property='children'),Output(component_id="my_bee_map",component_property="figure")],
[Input(component_id="select_year",component_property="value")])

## defione a function after callback, number of returns for a function is equal to the number of output defined above
def update_graph(option_slctd):## argument refers to the input component_property 
    print(option_slctd)
    print(type(option_slctd))
    container="The value choosen by the user is {}".format(option_slctd)
    dff=df.copy()
    dff=dff[dff['Year']==option_slctd]
    dff=dff[dff['Affected by']=='Varroa_mites']
    
    fig=ex.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark')
    
    return container,fig

In [30]:
if __name__=="__main__":
    app.run_server(port=4050,
        host='0.0.0.0')

Dash is running on http://0.0.0.0:4050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
